In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,minmax_scale,LabelEncoder

In [2]:
df =pd.read_csv("resourcesxxx.csv")


In [3]:
X = df.drop(columns=['DataCenterID'])
y = df['DataCenterID']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

noise_factor = 0.15  # You can adjust this parameter based on your needs
X_noisy = X_scaled + noise_factor * np.random.normal(size=X_scaled.shape)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2)


X_train_lstm = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_val_lstm = X_val.reshape(X_val.shape[0], 1, X_val.shape[1])

print(X_train_lstm.shape,X_val_lstm.shape)

(720, 1, 14) (180, 1, 14)


In [4]:
model = Sequential()
model.add(LSTM(units=50, input_shape=(1, X_train.shape[1]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(units=50))
model.add(BatchNormalization())  # Added Batch Normalization
model.add(Dense(units=len(df['DataCenterID'].unique()), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 50)             13000     
                                                                 
 dropout (Dropout)           (None, 1, 50)             0         
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 batch_normalization (Batch  (None, 50)                200       
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 3)                 153       
                                                                 
Total params: 33553 (131.07 KB)
Trainable params: 33453 (130.68 KB)
Non-trainable params: 100 (400.00 Byte)
____________

In [5]:

early_stopping = EarlyStopping(monitor='accuracy', patience=5, restore_best_weights=True)

history=model.fit(X_train_lstm, y_train, epochs=50, batch_size=19, validation_data=(X_val_lstm, y_val), callbacks=[early_stopping],verbose=2)


Epoch 1/50


38/38 - 9s - loss: 0.6386 - accuracy: 0.7694 - val_loss: 0.9996 - val_accuracy: 1.0000 - 9s/epoch - 241ms/step
Epoch 2/50
38/38 - 0s - loss: 0.1176 - accuracy: 0.9889 - val_loss: 0.9005 - val_accuracy: 0.9944 - 305ms/epoch - 8ms/step
Epoch 3/50
38/38 - 0s - loss: 0.0483 - accuracy: 0.9986 - val_loss: 0.7988 - val_accuracy: 1.0000 - 275ms/epoch - 7ms/step
Epoch 4/50
38/38 - 0s - loss: 0.0277 - accuracy: 0.9972 - val_loss: 0.6946 - val_accuracy: 1.0000 - 274ms/epoch - 7ms/step
Epoch 5/50
38/38 - 0s - loss: 0.0223 - accuracy: 1.0000 - val_loss: 0.5773 - val_accuracy: 1.0000 - 292ms/epoch - 8ms/step
Epoch 6/50
38/38 - 0s - loss: 0.0188 - accuracy: 0.9972 - val_loss: 0.4540 - val_accuracy: 1.0000 - 260ms/epoch - 7ms/step
Epoch 7/50
38/38 - 0s - loss: 0.0170 - accuracy: 0.9986 - val_loss: 0.3298 - val_accuracy: 1.0000 - 282ms/epoch - 7ms/step
Epoch 8/50
38/38 - 0s - loss: 0.0131 - accuracy: 0.9972 - val_loss: 0.2204 - val_accuracy: 1.0000 - 261ms/epoch - 7ms/step
Epoch 9/50
38/3

In [6]:
# model = Sequential()
# model.add(LSTM(units=128, return_sequences=True, input_shape=(1, X_train_lstm.shape[2])))
# model.add(LSTM(units=64, return_sequences=True))
# model.add(LSTM(units=64))
# model.add(Dense(units=1))
# model.compile(loss='accuracy', optimizer='adam')
# model.fit(X_train_lstm, y_train, epochs=10, batch_size=1, verbose=2)


In [7]:
plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

NameError: name 'plt' is not defined